In [ ]:
pip install tabulate

In [ ]:
## REORDERING DRAFT CLASS BASED ON ADVANCED STATS

import pandas as pd

# Loading and filtering dataset for specific year
df = pd.read_csv('your_dataset_path.csv')
df_2023 = df[df['Draft Year'] == 2023]

# Handling NaN values and reordering based on advanced stats
df_2023['WS_Reordering'] = df_2023['WS'].rank(method='max', ascending=False, na_option='keep')
df_2023['WS48_Reordering'] = df_2023['WS/48'].rank(method='max', ascending=False, na_option='keep')
df_2023['BPM_Reordering'] = df_2023['BPM'].rank(method='max', ascending=False, na_option='keep')
df_2023['VORP_Reordering'] = df_2023['VORP'].rank(method='max', ascending=False, na_option='keep')

# Preparing the final output
final_output = df_2023[['Pk', 'Tm', 'Player', 'WS_Reordering', 'WS48_Reordering', 'BPM_Reordering', 'VORP_Reordering']].copy()
final_output.rename(columns={'Pk': 'Pick', 'Tm': 'Team'}, inplace=True)
final_output = final_output.sort_values('Pick').reset_index(drop=True)

# Export the final output to CSV
final_output.to_csv('2023_NBA_Draft_Reordering_Full.csv', index=False)


In [ ]:
## CALCULATING SPEARMAN COEFFICIENTS FOR DRAFT CLASSES

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from scipy.stats import spearmanr
from tabulate import tabulate

# Loading dataset
df = pd.read_csv('/content/Testing DRAFT CLASSES.csv', encoding='utf-8')
df.fillna(-100, inplace=True)
results = []

# Loop through each year from 1984 to 2023
for year in range(1984, 2024):
    df_year = df[df['Draft Year'] == year].copy()

    # Handling NaN values and reordering based on advanced stats
    df_year['WS_Reordering'] = df_year['WS'].rank(method='max', ascending=False, na_option='keep')
    df_year['WS48_Reordering'] = df_year['WS/48'].rank(method='max', ascending=False, na_option='keep')
    df_year['BPM_Reordering'] = df_year['BPM'].rank(method='max', ascending=False, na_option='keep')
    df_year['VORP_Reordering'] = df_year['VORP'].rank(method='max', ascending=False, na_option='keep')

    # 'Average Rank' calculation
    df_year['Average_Rank'] = df_year[['WS_Reordering', 'WS48_Reordering', 'BPM_Reordering', 'VORP_Reordering']].mean(axis=1)
    final_output = df_year.rename(columns={'Pk': 'Pick', 'Tm': 'Team'})

    # Average Spearman Correlation calculation
    correlations = [
        spearmanr(final_output['Pick'], final_output['WS_Reordering'].fillna(-100)).correlation,
        spearmanr(final_output['Pick'], final_output['WS48_Reordering'].fillna(-100)).correlation,
        spearmanr(final_output['Pick'], final_output['BPM_Reordering'].fillna(-100)).correlation,
        spearmanr(final_output['Pick'], final_output['VORP_Reordering'].fillna(-100)).correlation
    ]
    spearman_corr_avg_rank = spearmanr(final_output['Pick'], final_output['Average_Rank']).correlation
    average_spearman = np.nanmean(correlations)
    results.append([year, average_spearman, spearman_corr_avg_rank])

# Displaying the results in table
results_df = pd.DataFrame(results, columns=['Year', 'Average Spearman Correlation', 'Spearman Correlation (Original vs. Average Rank)'])


## COLOR MAPPING
# Assign colors based on Spearman correlation values
def get_color(value):
    cmap = plt.cm.get_cmap('Blues')
    norm = mcolors.Normalize(-1, 1)
    return mcolors.to_hex(cmap(norm(value)))

results_df['Average Spearman Color'] = results_df['Average Spearman Correlation'].apply(get_color)
results_df['Spearman Correlation Color'] = results_df['Spearman Correlation (Original vs. Average Rank)'].apply(get_color)

results_df['Average Spearman Correlation'] = results_df['Average Spearman Correlation'].apply(lambda x: f"{x:.3f}")
results_df['Spearman Correlation (Original vs. Average Rank)'] = results_df['Spearman Correlation (Original vs. Average Rank)'].apply(lambda x: f"{x:.3f}")

results_df.to_csv('spearman_correlation_results_with_colors.csv', index=False)


In [ ]:
## CONVERTING TABLE TO LATEX CODE

import pandas as pd

# Load file
df = pd.read_csv('/content/Spearman Coefficients with Colors (1984 to 2023).csv')

latex_code = "\\begin{tabular}{|l|c|c|}\n\\hline\n"
latex_code += "\\rowcolor{gray!20}\n\\textbf{Year} & \\textbf{Average Spearman Corr.} & \\textbf{Spearman Corr. (Avg. Rank)} \\\\ \\hline\n"

# Looping through DataFrame
for index, row in df.iterrows():
    year = int(row['Year'])
    avg_corr = row['Average Spearman Correlation']
    rank_corr = row['Spearman Correlation (Average Rank)']
    # Check the values to determine the color of the cell
    avg_corr_color = 'lightgreen' if avg_corr >= 0.5 else 'lightred'
    rank_corr_color = 'lightgreen' if rank_corr >= 0.5 else 'lightred'
    latex_code += f"\\textbf{{{year}}} & \\cellcolor{{{avg_corr_color}}}{avg_corr:.3f} & \\cellcolor{{{rank_corr_color}}}{rank_corr:.3f} \\\\ \\hline\n"

latex_code += "\\end{tabular}\n"

with open('spearman_coefficients_table.txt', 'w') as file:
    file.write(latex_code)
